In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
import matplotlib.colors

%matplotlib qt

In [2]:
ghcn_file = 'gistemp1200_GHCNv4_ERSSTv5.nc'
ds = xr.open_dataset(ghcn_file)
dates = ds['time'].values
print(f'Total number of dates: {len(dates)}')

Total number of dates: 1723


In [3]:
# Create dictionary for plotting individual months
# (plotting all points would take too long)
month_dict = {}
month_dict['Jan'] = 0
month_dict['Feb'] = 1
month_dict['Mar'] = 2
month_dict['Apr'] = 3
month_dict['May'] = 4
month_dict['Jun'] = 5
month_dict['Jul'] = 6
month_dict['Aug'] = 7
month_dict['Sep'] = 8
month_dict['Oct'] = 9
month_dict['Nov'] = 10
month_dict['Dec'] = 11

In [4]:
# Create color map based on IPCC visual style guide
# https://www.ipcc.ch/site/assets/uploads/2019/04/IPCC-visual-style-guide.pdf
# Temperature section, page 10

# Set RGB values (using 5 color scheme)
red = [202, 0, 32]
orange = [244, 165, 130]
white = [247, 247, 247]
light_blue = [146, 197, 222]
blue = [5, 113, 176]

# Set RGB values (using 11 color scheme)
red1 = [103, 0, 31]
red2 = [178, 24, 43]
red3 = [214, 96, 77]
red4 = [244, 165, 130]
red5 = [253, 219, 199]
white = [247, 247, 247]
blue5 = [209, 229, 240]
blue4 = [146, 197, 222]
blue3 = [67, 147, 195]
blue2 = [33, 102, 172]
blue1 = [5, 48, 97]

# Normalize values
color_list_5 = [blue, light_blue, white, orange, red]
color_list_11 = [blue1, blue2, blue3, blue4, blue5, white, red5, red4, red3, red2, red1]
color_tuples = []
for color in color_list_11:
    color_tuples.append([x/256 for x in color])

# Create cmap for later plotting
IPCC_cmap = matplotlib.colors.LinearSegmentedColormap.from_list('IPCC_RGB', color_tuples)

In [5]:
def spatial_temp_plot(month='Jan'):

    # Collect all dates for given month
    month_dates = []
    for i in range(len(dates)):
        if i % 12 == month_dict[month]:
            month_dates.append(dates[i])

    # Initialize figure
    fig = plt.figure(figsize=(10,6))

    # Set counter and min/max values
    counter = 0
    max_val = 2
    min_val = -max_val
    num_points = len(month_dates) - 1

    # Loop through all dates
    while True:

        # Plot temperature anomaly for each month
        data = ds['tempanomaly'].sel(time=month_dates[counter])
        ax = plt.axes(projection=ccrs.Robinson())
        ax.coastlines()
        ax.gridlines()
        data.plot.pcolormesh(ax=ax,
                  cmap=IPCC_cmap,
                  transform=ccrs.PlateCarree(),
                  cbar_kwargs={'orientation':'horizontal', 'pad':0.1},
                  vmin=min_val,
                  vmax=max_val)
        plt.draw()
        date = str(data.time.values).split('T')[0]
        title = 'GISTEMP Global Temperature Anomaly\n' + date
        plt.title(title)
        plt.pause(10**(-3))

        # Clear plot to make room for next one
        if counter < num_points:
            plt.clf()

        # Stop plotting if last date reached
        else:
            break

        # Increment counter
        counter += 1

In [6]:
spatial_temp_plot('Oct')